In [1]:
from module.prepare import *
from itertools import product
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

import gc
import os
import re
import math
import sys
from collections import Counter
import random
from itertools import islice
import time
import configparser
import json
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import bokeh

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy as sp
from scipy import stats

import sklearn
from joblib import dump, load
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.manifold import *
import sklearn.tree as Tr 

import lightgbm as lgb

D:\IDE_Project_Programming\Anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
def getCurrentTime():
    return datetime.datetime.strftime(datetime.datetime.fromtimestamp(time.time()),format='%Y-%m-%d-%H-%M-%S')


def LGBTuning(Xtrain,Xtest,Ytrain,Ytest,new_params=None):
    
    clf = lgb.LGBMClassifier(objective='cross_entropy', ### {cross_entropy, binary}
                             silent=False,
                             verbose=1,
                             random_state=seed,
                             n_jobs=12,
#                              class_weight
                            )
    
    default_params = {
    'learning_rate': [0.1], 
    'boosting_type':['gbdt'], 
    'n_estimators': [500],
    'num_iterations':[1000],
    'max_bin':[256]
    }
    
    if new_params is not None:
        gridParams.update(new_params)
    
    arg_str = ''
    for k,v in default_params.items():
        if type(v[0])==str:
            arg_str += k+'='+"'"+v[0]+"',"
        else:
            arg_str += k+'='+str(v[0])+","
    eval(
        'clf.'+clf.set_params.__name__+"("
            +arg_str.rstrip(',')+
            ")"
        )

    print('DEBUG:: tuning params\n',clf.get_params())
    clf.fit(Xtrain,Ytrain)
    Ypred = clf.predict(Xtest)
    score_train = clf.score(Xtrain,Ytrain)
    print('train score %f'%score_train)
    
    return [Ypred,Ytest,score_train,clf]

In [3]:
cv = 5
generalize_ratio = 1.0/cv
test_ratio = 1.0/cv
tuning_mode = False

if tuning_mode:
    cv = 1

tuning 1

In [4]:
def TuningParametersStage1(fname=getCurrentTime()+'-stage1.csv'):
    res = []
    for DATAID in [0,1,2,3,4]:
        INFO('data id %d'%DATAID)
        for RNA_K in range(3,7):
            for PROTEIN_K in range(3,7):
                for TOP_RATIO in np.linspace(0.93,0.99,5):
                    [data,T] = ReadData(DATAID,PROTEIN_K,RNA_K)
                    [X,Y] = ToMatrix(data,'dense')
                    [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
                    [X_train,X_test,Y_train,Y_test] = \
                                RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test,topRatio=0.96)
                    r = LGBTuning(X_train,X_test,Y_train,Y_test,tuning=False,default=True)
                    r = {
                        'DATAID': DATAID,
                        'test_score':scoreFunction(r[0],r[1]),
                        'train_score':r[2],
                        'RNA_K':RNA_K,
                        'PROTEIN_K':PROTEIN_K,
                        'TOP_RATIO':TOP_RATIO
                    }
                    res.append(r)
    df = pd.DataFrame(data=res,columns=['DATAID','test_score','train_score','RNA_K','PROTEIN_K','TOP_RATIO'])
    df.to_csv(os.path.join('./result',fname))
    return


In [ ]:
TuningParametersStage1()